## IEEE 754 Double Precision (64-bit) Floating Point Converter
Math 570 Group Project

This module implements conversion between real numbers and 64-bit 
floating-point representations following the IEEE 754 standard.


In [ ]:
import math

In [ ]:
def real_to_float64_chopping(number):
    """
    Convert real number to 64-bit IEEE 754 using CHOPPING
    
    Args:
        number (float): Real number to convert
    
    Returns:
        str: 64-bit binary string representation
    """
    # Special case: zero
    if number == 0.0:
        return '0' * 64
    
    # Special case: infinity
    if math.isinf(number):
        if number > 0:
            return '0' + '1'*11 + '0'*52
        else:
            return '1' + '1'*11 + '0'*52
    
    # Special case: NaN
    if math.isnan(number):
        return '0' + '1'*11 + '1'*52
    
    # Extract sign
    sign = '1' if number < 0 else '0'
    number = abs(number)
    
    # Normalize to get exponent
    exponent = 0
    temp = number
    
    if temp >= 2:
        while temp >= 2:
            temp /= 2
            exponent += 1
    elif temp < 1:
        while temp < 1:
            temp *= 2
            exponent -= 1
    
    # Biased exponent (bias = 1023 for double precision)
    biased_exp = exponent + 1023
    
    # Check for overflow/underflow
    if biased_exp >= 2047:
        # Overflow to infinity
        return sign + '1'*11 + '0'*52
    if biased_exp <= 0:
        # Underflow to zero (simplified)
        return sign + '0'*63
    
    exp_bits = format(biased_exp, '011b')
    
    # Extract mantissa (52 bits with chopping)
    mantissa = temp - 1.0  # Remove implicit leading 1
    mantissa_bits = ''
    
    for i in range(52):
        mantissa *= 2
        if mantissa >= 1:
            mantissa_bits += '1'
            mantissa -= 1
        else:
            mantissa_bits += '0'
    
    return sign + exp_bits + mantissa_bits


In [ ]:
def real_to_float64_rounding(number):
    """
    Convert real number to 64-bit IEEE 754 using ROUNDING
    
    Args:
        number (float): Real number to convert
    
    Returns:
        str: 64-bit binary string representation
    """
    # Special case: zero
    if number == 0.0:
        return '0' * 64
    
    # Special case: infinity
    if math.isinf(number):
        if number > 0:
            return '0' + '1'*11 + '0'*52
        else:
            return '1' + '1'*11 + '0'*52
    
    # Special case: NaN
    if math.isnan(number):
        return '0' + '1'*11 + '1'*52
    
    # Extract sign
    sign = '1' if number < 0 else '0'
    number = abs(number)
    
    # Normalize to get exponent
    exponent = 0
    temp = number
    
    if temp >= 2:
        while temp >= 2:
            temp /= 2
            exponent += 1
    elif temp < 1:
        while temp < 1:
            temp *= 2
            exponent -= 1
    
    # Biased exponent
    biased_exp = exponent + 1023
    
    # Check for overflow/underflow
    if biased_exp >= 2047:
        return sign + '1'*11 + '0'*52
    if biased_exp <= 0:
        return sign + '0'*63
    
    # Extract mantissa (53 bits for rounding decision)
    mantissa = temp - 1.0
    mantissa_bits = ''
    
    for i in range(53):
        mantissa *= 2
        if mantissa >= 1:
            mantissa_bits += '1'
            mantissa -= 1
        else:
            mantissa_bits += '0'
    
    # Apply rounding
    if mantissa_bits[52] == '1':
        # Round up
        mantissa_int = int(mantissa_bits[:52], 2) + 1
        if mantissa_int >= 2**52:
            # Mantissa overflow
            biased_exp += 1
            mantissa_bits = '0' * 52
        else:
            mantissa_bits = format(mantissa_int, '052b')
    else:
        mantissa_bits = mantissa_bits[:52]
    
    exp_bits = format(biased_exp, '011b')
    return sign + exp_bits + mantissa_bits


In [ ]:
def float64_to_real(bit_string):
    """
    Convert 64-bit IEEE 754 representation to real number
    
    Args:
        bit_string (str): 64-bit binary string
    
    Returns:
        float: Real number representation
    """
    # Validate input
    if len(bit_string) != 64:
        raise ValueError(f"Input must be exactly 64 bits, got {len(bit_string)}")
    
    if not all(c in '01' for c in bit_string):
        raise ValueError("Input must contain only 0s and 1s")
    
    # Extract components
    sign_bit = bit_string[0]
    exp_bits = bit_string[1:12]
    mantissa_bits = bit_string[12:64]
    
    # Convert to values
    sign = -1 if sign_bit == '1' else 1
    exponent = int(exp_bits, 2)
    
    # Special cases
    if exponent == 2047:  # All 1s in exponent
        if mantissa_bits == '0' * 52:
            return sign * float('inf')
        else:
            return float('nan')
    
    if exponent == 0:  # Zero or denormalized
        if mantissa_bits == '0' * 52:
            return 0.0
        else:
            # Denormalized number
            mantissa_value = 0.0
            for i, bit in enumerate(mantissa_bits):
                if bit == '1':
                    mantissa_value += 2 ** (-(i + 1))
            return sign * mantissa_value * (2 ** -1022)
    
    # Normalized number
    mantissa_value = 1.0  # Implicit leading 1
    for i, bit in enumerate(mantissa_bits):
        if bit == '1':
            mantissa_value += 2 ** (-(i + 1))
    
    # Calculate final value
    real_exponent = exponent - 1023
    result = sign * mantissa_value * (2 ** real_exponent)
    
    return result


In [ ]:
def display_components(bit_string):
    """
    Display the components of a 64-bit floating point number
    
    Args:
        bit_string (str): 64-bit binary string
    """
    if len(bit_string) != 64:
        print("Error: Input must be 64 bits")
        return
    
    sign = bit_string[0]
    exponent = bit_string[1:12]
    mantissa = bit_string[12:64]
    
    print(f"Sign bit:     {sign}")
    print(f"Exponent:     {exponent} (decimal: {int(exponent, 2)})")
    print(f"Mantissa:     {mantissa}")
    print(f"Full binary:  {bit_string}")


In [ ]:
# Test cases
if __name__ == "__main__":
    print("=" * 70)
    print("IEEE 754 Double Precision Floating Point Converter")
    print("=" * 70)
    
    # Test cases
    test_numbers = [0.0, 1.0, -1.0, 12.375, -12.375, 0.1, 
                    float('inf'), float('-inf'), float('nan')]
    
    print("\n--- CHOPPING METHOD ---")
    for num in test_numbers[:7]:  # Skip inf and nan for comparison
        bits_chop = real_to_float64_chopping(num)
        recovered = float64_to_real(bits_chop)
        print(f"\nOriginal:  {num}")
        display_components(bits_chop)
        print(f"Recovered: {recovered}")
        print(f"Error:     {abs(num - recovered) if not math.isnan(num) else 0}")
    
    print("\n\n--- ROUNDING METHOD ---")
    for num in test_numbers[:7]:
        bits_round = real_to_float64_rounding(num)
        recovered = float64_to_real(bits_round)
        print(f"\nOriginal:  {num}")
        display_components(bits_round)
        print(f"Recovered: {recovered}")
        print(f"Error:     {abs(num - recovered) if not math.isnan(num) else 0}")
    
    print("\n\n--- COMPARISON: Chopping vs Rounding ---")
    test_val = 12.375
    bits_chop = real_to_float64_chopping(test_val)
    bits_round = real_to_float64_rounding(test_val)
    print(f"Number: {test_val}")
    print(f"Chopping:  {bits_chop}")
    print(f"Rounding:  {bits_round}")
    print(f"Same result: {bits_chop == bits_round}")
